In [5]:
import numpy as np
import random
from qiskit import *
from qiskit import BasicAer
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram
from qiskit.aqua import QuantumInstance
from qiskit.aqua.components.optimizers import SPSA

In [2]:
def create_edge_matrix(number_of_points, probability):
    edge_matrix = np.ones((number_of_points,number_of_points))
    for i in range(0,number_of_points):
        for j in range(i,number_of_points):
            if random.random()>probability or i==j:
                edge_matrix[i,j]=0
                edge_matrix[j,i]=0
    return edge_matrix

create_edge_matrix(5, 0.5)

array([[0., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0.],
       [0., 1., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.]])

In [12]:
def Create_quadratic_program(n, edge_matrix):
    qubo = QuadraticProgram()
    linear_coefficients = list()
    summation_terms = dict()
    for i in range(n):
        qubo.binary_var('x'+ str(i))
        linear_coefficients.append(1)
    for i in range(n):
        for j in range(i,n):
            if edge_matrix[i,j] == 0 and i!=j:
                summation_terms[('x'+str(i),'x'+str(j))] = summation_terms.get(('x'+str(i),'x'+str(j)), -4*n)
    qubo.maximize(linear= linear_coefficients, quadratic= summation_terms)
    #print(qubo.export_as_lp_string())
    op, offset = qubo.to_ising()
    return op, offset, qubo
Create_quadratic_program(5, create_edge_matrix(5,0.5))

(SummedOp([PauliOp(Pauli(z=[True, False, False, False, False], x=[False, False, False, False, False]), coeff=-14.5), PauliOp(Pauli(z=[False, True, False, False, False], x=[False, False, False, False, False]), coeff=-14.5), PauliOp(Pauli(z=[False, False, True, False, False], x=[False, False, False, False, False]), coeff=-9.5), PauliOp(Pauli(z=[False, False, False, True, False], x=[False, False, False, False, False]), coeff=-4.5), PauliOp(Pauli(z=[False, False, False, False, True], x=[False, False, False, False, False]), coeff=-4.5), PauliOp(Pauli(z=[True, True, False, False, False], x=[False, False, False, False, False]), coeff=5.0), PauliOp(Pauli(z=[True, False, True, False, False], x=[False, False, False, False, False]), coeff=5.0), PauliOp(Pauli(z=[False, True, True, False, False], x=[False, False, False, False, False]), coeff=5.0), PauliOp(Pauli(z=[True, False, False, True, False], x=[False, False, False, False, False]), coeff=5.0), PauliOp(Pauli(z=[False, True, False, False, True],

In [37]:
def cost_qaoa(n, edge_matrix):
    operator, offset, qubo = Create_quadratic_program(n, edge_matrix)
    seed = 10598
    backend = Aer.get_backend('statevector_simulator')
    quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed)
    spsa = SPSA(maxiter=300)
    qaoa_API = QAOA(operator, spsa, p = 4, max_evals_grouped = 1 ,quantum_instance = quantum_instance )
    result = QAOA.find_minimum(qaoa_API)
    cost = (-1*result['optimal_value'])
    tim = result['optimizer_time']
    return cost, tim
cost_qaoa(5, create_edge_matrix(5,0.5))

(-8.823985238872961, 4.9344704151153564)

In [49]:
def result_exact(qubo):
    exact_mes = NumPyMinimumEigensolver()
    exact = MinimumEigenOptimizer(exact_mes)
    exact_result = exact.solve(qubo)
    print(exact_result)
op, offset, qubo = Create_quadratic_program(16, create_edge_matrix(16,0.5))
result_exact(qubo)

############ working fine till 16 only ##############

optimal function value: 5.0
optimal value: [0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1.]
status: SUCCESS


In [51]:
def result_qaoa(qubo):
    qaoa_mes = QAOA(quantum_instance=BasicAer.get_backend('statevector_simulator'))
    qaoa = MinimumEigenOptimizer(qaoa_mes)
    qaoa_result = qaoa.solve(qubo)
    print(qaoa_result)
op, offset, qubo = Create_quadratic_program(13, create_edge_matrix(13,0.5))
result_qaoa(qubo)

optimal function value: 4.0
optimal value: [0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]
status: SUCCESS
